In [25]:
import glob
import pandas as pd

# Baby Names by State

In [26]:
dirpath = "./namesbystate/"
csv_files = glob.glob(dirpath + "/*.TXT")

df_list = []
for f in csv_files:
    df_list.append(pd.read_csv(f, header=None, 
                               names=["state", "gender", "year", "name", "occurrences"]))

In [27]:
big_df = pd.concat(df_list, ignore_index=True, axis=0)
len(big_df)

6408041

In [28]:
big_df.head(10)

,state,gender,year,name,occurrences
0,ID,F,1910,Mary,53
1,ID,F,1910,Dorothy,31
2,ID,F,1910,Helen,30
3,ID,F,1910,Margaret,24
4,ID,F,1910,Ruth,24
5,ID,F,1910,Gladys,20
6,ID,F,1910,Alice,19
7,ID,F,1910,Mildred,19
8,ID,F,1910,Thelma,19
9,ID,F,1910,Edith,17


In [29]:
big_df.to_csv("./output/namesbystate_combined.csv")

In [30]:
# TODO: assign rankings?

In [31]:
subset_df = big_df.groupby(["year", "state", "gender"]).apply(
    lambda x: x.sort_values(by="occurrences", ascending=False).head(1)).copy()

In [32]:
subset_df.to_csv("./output/subset_df.csv")

In [33]:
subset_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11526 entries, (1910, 'AK', 'F', 2838142) to (2022, 'WY', 'M', 4598904)
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   state        11526 non-null  object
 1   gender       11526 non-null  object
 2   year         11526 non-null  int64 
 3   name         11526 non-null  object
 4   occurrences  11526 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 862.3+ KB


# Exploring popularity of "-den" names

In [34]:
big_df[(big_df["name"].str.endswith("den")) | \
       (big_df["name"].str.endswith("dan")) | \
       (big_df["name"].str.endswith("dyn")) \
      ].sort_values(by="occurrences", ascending=False).head(20)

,state,gender,year,name,occurrences
3292085,CA,M,2012,Jayden,2871
3289196,CA,M,2011,Jayden,2754
3295001,CA,M,2013,Jayden,2581
3286287,CA,M,2010,Jayden,2444
3297902,CA,M,2014,Jayden,2383
3283365,CA,M,2009,Jayden,2263
3300842,CA,M,2015,Jayden,2134
3280458,CA,M,2008,Jayden,2055
3292096,CA,M,2012,Aiden,2003
3286298,CA,M,2010,Aiden,1941


# All Baby Names by Year and Gender

In [35]:
import re

dirpath = "./names/"
csv_files = glob.glob(dirpath + "/*.txt")

regex_str = re.compile(f"yob(\d\d\d\d).txt")

df_list = []
for f in csv_files:
    year = re.search(regex_str, f).groups()[0]
    temp_df = pd.read_csv(
        f, header=None, 
        names=["name", "gender", "occurrences"])
    temp_df.insert(1, "year", year)
    df_list.append(temp_df)

print(df_list)

[          name  year gender  occurrences
0        Emily  2000      F        25957
1       Hannah  2000      F        23085
2      Madison  2000      F        19968
3       Ashley  2000      F        17998
4        Sarah  2000      F        17712
...        ...   ...    ...          ...
29771     Zeph  2000      M            5
29772    Zeven  2000      M            5
29773    Ziggy  2000      M            5
29774       Zo  2000      M            5
29775    Zyier  2000      M            5

[29776 rows x 4 columns],            name  year gender  occurrences
0          Emma  2014      F        20958
1        Olivia  2014      F        19829
2        Sophia  2014      F        18636
3      Isabella  2014      F        17125
4           Ava  2014      F        15721
...         ...   ...    ...          ...
33289    Zykeem  2014      M            5
33290    Zymeer  2014      M            5
33291   Zymiere  2014      M            5
33292     Zyran  2014      M            5
33293     Zyrin  2

In [36]:
big_df = pd.concat(df_list, ignore_index=True, axis=0)

# Assessing Most Popular and Most Unique Baby Names

In [37]:
highest_df = big_df.groupby(["year", "gender"]).apply(
    lambda x: x[x["occurrences"] == x["occurrences"].max()]).copy()

In [38]:
", ".join(sorted(highest_df["name"].unique()))

'Ashley, David, Emily, Emma, Isabella, Jacob, James, Jennifer, Jessica, John, Liam, Linda, Lisa, Mary, Michael, Noah, Olivia, Robert, Sophia'

In [39]:
highest_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 286 entries, ('1880', 'F', 1935794) to ('2022', 'M', 1849802)
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         286 non-null    object
 1   year         286 non-null    object
 2   gender       286 non-null    object
 3   occurrences  286 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 26.1+ KB


In [40]:
highest_df.to_csv("./output/most_popular_names_1880_2022.csv")

## Flourish connected scatter plot
<a href='https://public.flourish.studio/visualisation/14251422/'>Most Popular Baby Names</a>


In [41]:
lowest_df = big_df.groupby(["year", "gender"]).apply(
    lambda x: x[x["occurrences"] == x["occurrences"].min()]).copy()

In [42]:
lowest_df_counts_df = lowest_df["name"].value_counts().reset_index(name='Counts')

In [43]:
lowest_df_counts_df.to_csv("./output/least_popular_names_1880_2022.csv")

## Flourish table
<a href='https://public.flourish.studio/visualisation/14251690/'>Most Unique Baby Names</a>